In [37]:
#import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [38]:
# Load the dataset
df = pd.read_csv("assignment-data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103392 entries, 0 to 103391
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            103392 non-null  int64  
 1   datetime              103392 non-null  object 
 2   load                  101952 non-null  float64
 3   apparent_temperature  103392 non-null  float64
 4   temperature           103392 non-null  float64
 5   humidity              103392 non-null  float64
 6   dew_point             103392 non-null  float64
 7   wind_speed            103392 non-null  float64
 8   cloud_cover           103392 non-null  float64
 9   date                  103392 non-null  object 
dtypes: float64(7), int64(1), object(2)
memory usage: 7.9+ MB


In [39]:
df.isnull().sum() # checking for null values

,0
Unnamed: 0,0
datetime,0
load,1440
apparent_temperature,0
temperature,0
humidity,0
dew_point,0
wind_speed,0
cloud_cover,0
date,0


In [40]:
# Convert 'datetime' to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])

# Drop the unnecessary 'Unnamed: 0' column
df.drop(columns=["Unnamed: 0"], inplace=True)

# Handle missing values in 'load' using interpolation
df["load"] = df["load"].interpolate(method="linear")


In [41]:
df.isnull().sum()

,0
datetime,0
load,0
apparent_temperature,0
temperature,0
humidity,0
dew_point,0
wind_speed,0
cloud_cover,0
date,0


In [42]:
# Extract time-based features
df["hour"] = df["datetime"].dt.hour
df["day"] = df["datetime"].dt.day
df["month"] = df["datetime"].dt.month
df["weekday"] = df["datetime"].dt.weekday  # Monday=0, Sunday=6

# Define a list of Indian national holidays (2018-2020)
holidays = [
    "2018-08-15", "2018-10-02", "2018-12-25",
    "2019-08-15", "2019-10-02", "2019-12-25",
    "2020-08-15", "2020-10-02", "2020-12-25"
]

# Convert to datetime and create a holiday feature
df["is_holiday"] = df["datetime"].dt.date.astype(str).isin(holidays).astype(int)

# Verify new features
print(df.head())


             datetime       load  apparent_temperature  temperature  humidity  \
0 2018-01-01 00:00:00  803.22270              10.45800     10.45800  0.955500   
1 2018-01-01 00:15:00  774.89523              10.32675     10.32675  0.961625   
2 2018-01-01 00:30:00  731.46927              10.19550     10.19550  0.967750   
3 2018-01-01 00:45:00  713.93870              10.06425     10.06425  0.973875   
4 2018-01-01 01:00:00  699.23007               9.93300      9.93300  0.980000   

   dew_point  wind_speed  cloud_cover        date  hour  day  month  weekday  \
0   8.946000         0.0          0.0  2018-01-01     0    1      1        0   
1   8.911875         0.0          0.0  2018-01-01     0    1      1        0   
2   8.877750         0.0          0.0  2018-01-01     0    1      1        0   
3   8.843625         0.0          0.0  2018-01-01     0    1      1        0   
4   8.809500         0.0          0.0  2018-01-01     1    1      1        0   

   is_holiday  
0           0  


In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # Import mean_absolute_percentage_error
# Select features and target
features = ["hour", "day", "month", "weekday", "temperature", "humidity", "wind_speed", "is_holiday"]
X = df[features]
y = df["load"]

# Split data: Use past data for training, exclude December 2020
X_train = X[df["datetime"] < "2020-12-01"]
y_train = y[df["datetime"] < "2020-12-01"]

X_test = X[df["datetime"] >= "2020-12-01"]
y_test = y[df["datetime"] >= "2020-12-01"]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost Model
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, objective="reg:squarederror")
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Mean Absolute Percentage Error (MAPE): 5.73%


In [44]:

# Generate 15-minute interval timestamps for Dec 14, 2020
date_14_dec = pd.date_range(start="2020-12-14 00:00:00", end="2020-12-14 23:45:00", freq="15T")

#  Create a DataFrame with these timestamps
forecast_df = pd.DataFrame({"datetime": date_14_dec})

#  Add the same feature columns as in training
forecast_df["hour"] = forecast_df["datetime"].dt.hour
forecast_df["day"] = forecast_df["datetime"].dt.day
forecast_df["month"] = forecast_df["datetime"].dt.month
forecast_df["weekday"] = forecast_df["datetime"].dt.weekday  # Monday=0, Sunday=6

#  Since we don't have actual weather data for Dec 14, 2020, use the average of past Dec 14 values
weather_features = ["temperature", "humidity", "wind_speed"]
for feature in weather_features:
    forecast_df[feature] = df[df["month"] == 12][feature].mean()

#  Add holiday indicator
forecast_df["is_holiday"] = 1 if "2020-12-14" in ["2020-08-15", "2020-10-02", "2020-12-25"] else 0

#  Drop 'datetime' before scaling
X_dec14 = forecast_df.drop(columns=["datetime"])

#  Scale the features
X_dec14_scaled = scaler.transform(X_dec14)

#  Predict load values
forecast_df["forecast"] = model.predict(X_dec14_scaled)

#  Save to CSV
forecast_df.to_csv("forecast_dec14_xgboost.csv", index=False)

print(" Forecast for Dec 14, 2020 saved as 'forecast_dec14_xgboost.csv'")

 Forecast for Dec 14, 2020 saved as 'forecast_dec14_xgboost.csv'


<ipython-input-44-9828cbde0884>:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_14_dec = pd.date_range(start="2020-12-14 00:00:00", end="2020-12-14 23:45:00", freq="15T")


In [45]:
from google.colab import files
files.download("forecast_dec14_xgboost.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
# Select only datetime and forecast columns
final_forecast_df = forecast_df[["datetime", "forecast"]]

# Save the final CSV file for submission
final_forecast_df.to_csv("final_forecast_dec14.csv", index=False)

print("Final submission file saved as 'final_forecast_dec14.csv'")

Final submission file saved as 'final_forecast_dec14.csv'


In [47]:
from google.colab import files
files.download("final_forecast_dec14.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>